In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [5]:
# Step 1: Load dataset
import pandas as pd

DATA_PATH = "/content/fashion_dataset.csv"  # change if needed
df = pd.read_csv(DATA_PATH)

print("Rows, cols:", df.shape)
df.head()


Rows, cols: (200, 8)


,ID,Skin Tone,Weather Condition,Work Level,Season,Recommended Outfit,Fabric,Avoid Fabrics
0,1,Medium,Hot,Home,Summer,Relaxed light blouse,Rayon,Wool
1,2,Olive,Hot,Home,Spring,Relaxed light jacket,Cotton Blend,Fleece
2,3,Light,Rainy,Fitness,Spring,Active pastel shirt,Denim,Velvet
3,4,Fair,Windy,Outdoor,Summer,Casual shorts,Rayon,Wool
4,5,Light,Hot,Home,Summer,Relaxed linen shirt,Cotton,Polyester


In [8]:
df['combined_text'] = (
    df['Skin Tone'] + " " +
    df['Weather Condition'] + " " +
    df['Work Level'] + " " +
    df['Season']

)


In [9]:
X_text = df['combined_text']
y = df['Recommended Outfit']  # or 'Avoid Fabrics' if you prefer predicting that
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
X = vectorizer.fit_transform(X_text)

print("TF-IDF Matrix Shape:", X.shape)

TF-IDF Matrix Shape: (200, 112)


In [10]:
# ============================================================
# 3️⃣ TRAIN TEST SPLIT
# ============================================================
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train:", X_train.shape, " Test:", X_test.shape)

Train: (160, 112)  Test: (40, 112)


In [11]:
# ============================================================
# 4️⃣ MODEL TRAINING
# ============================================================
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000, multi_class='multinomial')
model.fit(X_train, y_train)

print("✅ Model training complete!")


✅ Model training complete!


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [12]:
# ============================================================
# 5️⃣ EVALUATION
# ============================================================
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc*100:.2f}%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 27.50%

Classification Report:
                           precision    recall  f1-score   support

         Active cardigan       0.00      0.00      0.00         2
   Active cotton t-shirt       0.00      0.00      0.00         1
     Active light blouse       0.00      0.00      0.00         1
      Active linen shirt       0.33      1.00      0.50         1
     Active pastel shirt       0.33      1.00      0.50         1
         Active raincoat       0.00      0.00      0.00         1
      Active scarf combo       0.00      0.00      0.00         1
   Active thermal jacket       0.50      1.00      0.67         1
Active waterproof jacket       0.50      1.00      0.67         1
     Active wool sweater       0.00      0.00      0.00         1
            Casual boots       0.00      0.00      0.00         1
         Casual cardigan       0.00      0.00      0.00         2
             Casual coat       0.67      1.00      0.80         2
     Casual denim jacket       0.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_

In [13]:
# ============================================================
# 6️⃣ SAVE MODEL AND VECTORIZER
# ============================================================
with open("fashion_fabric_model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("fashion_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

print("\n✅ Model and vectorizer saved successfully as .pkl files!")



✅ Model and vectorizer saved successfully as .pkl files!


In [14]:
# ============================================================
# 7️⃣ CUSTOM PREDICTION FUNCTION
# ============================================================
def predict_fabric(skin_tone, weather, work_level, season, outfit_desc):
    text = f"{skin_tone} {weather} {work_level} {season} {outfit_desc}"
    vector = vectorizer.transform([text])
    pred = model.predict(vector)[0]
    print(f"\n🧾 Input: {text}")
    print(f"💡 Predicted Fabric: {pred}")

# Test Examples
predict_fabric("Fair", "Sunny", "Office", "Summer", "white cotton shirt")
predict_fabric("Deep", "Cold", "Outdoor", "Winter", "wool jacket and gloves")



🧾 Input: Fair Sunny Office Summer white cotton shirt
💡 Predicted Fabric: Formal linen shirt

🧾 Input: Deep Cold Outdoor Winter wool jacket and gloves
💡 Predicted Fabric: Casual coat


In [15]:
import pickle

file_path = '/content/fashion_fabric_model.pkl'

try:
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    print(data)
except FileNotFoundError:
    print(f"File not found: {file_path}")
except pickle.UnpicklingError:
    print("Error: The file content is not a valid pickle format.")
except EOFError:
    print("Error: The file is incomplete or corrupted.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

LogisticRegression(max_iter=1000, multi_class='multinomial')


In [16]:
import pickle

# Load the vectorizer
with open('/content/fashion_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

print(type(vectorizer))
print(vectorizer)

# Load the model
with open('/content/fashion_fabric_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Example usage (assuming X_test is available from previous cells)
y_pred = model.predict(X_test)
print("\nExample prediction using loaded model:")
print(y_pred[:5]) # Print the first 5 predictions

<class 'sklearn.feature_extraction.text.TfidfVectorizer'>
TfidfVectorizer(ngram_range=(1, 2), stop_words='english')

Example prediction using loaded model:
['Relaxed raincoat' 'Casual raincoat' 'Active pastel shirt'
 'Formal linen shirt' 'Formal linen shirt']
